# Semana 1 do challenge

## Tarefa 1 - Entender quais informações o conjunto de dados possui

### Dicionário de dados

* `customerID`: número de identificação único de cada cliente
* `Churn`: se o cliente deixou ou não a empresa 
* `gender`: gênero (masculino e feminino) 
* `SeniorCitizen`: informação sobre um cliente ter ou não idade igual ou maior que 65 anos 
* `Partner`:  se o cliente possui ou não um parceiro ou parceira
* `Dependents`: se o cliente possui ou não dependentes
* `tenure`:  meses de contrato do cliente
* `PhoneService`: assinatura de serviço telefônico 
* `MultipleLines`: assisnatura de mais de uma linha de telefone 
* `InternetService`: assinatura de um provedor internet 
* `OnlineSecurity`: assinatura adicional de segurança online 
* `OnlineBackup`: assinatura adicional de backup online 
* `DeviceProtection`: assinatura adicional de proteção no dispositivo 
* `TechSupport`: assinatura adicional de suporte técnico, menos tempo de espera
* `StreamingTV`: assinatura de TV a cabo 
* `StreamingMovies`: assinatura de streaming de filmes 
* `Contract`: tipo de contrato
* `PaperlessBilling`: se o cliente prefere receber online a fatura
* `PaymentMethod`: forma de pagamento
* `Charges.Monthly`: total de todos os serviços do cliente por mês
* `Charges.Total`: total gasto pelo cliente

### Leitura inicial dos dados

In [278]:
import pandas as pd
pd.set_option("display.max_columns", 100)

dados = pd.read_json(
    path_or_buf='Telco-Customer-Churn.json',
    orient='columns'
)
dados.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


O dataset possui informaçções gerais do cliente como ID e seu Churn. Há ainda outras quatro listas referentes a caracerísticas do cliente. 
* customer: Caractéristicas gerais do cliente
* phone: Informações sobre a conta telefônica do cliente
* internet: Informações da conta de internet do cliente
* account: Informações do contrato do cliente

### Extraindo conteúdo das colunas

O arquivo json será normalizado para um DataFrame do pandas.

In [279]:
# Extraindo dados das listas do arquivo json

customer = pd.json_normalize(data=dados.customer)
phone = pd.json_normalize(data=dados.phone)
internet = pd.json_normalize(data=dados.internet)
account = pd.json_normalize(data=dados.account, sep='_')

Concatenando os dados em um único DataFrame

In [280]:
dados = pd.concat([dados[['customerID', 'Churn']], customer, phone, internet, account], axis=1) 

## Tarefa 2 - Analisar quais os tipos de dados

Informações sobre as variáveis.

In [281]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


O dataset contém 20 variáveis. Automaticamente a coluna `ternure`foi reconhedida como do tipo inteiro e a coluna `Charges_Monthly` como do tipo float64 as demais foram reconhecidas como object. Porém, a coluna `Charges_Total` também deverá ser reconhecida como do tipo float64, esta correção é feita na próxima secção. 

## Tarefa 3 - Verificar quais as inconsistências nos dados

Na coluna `Charges_Total` existem células preenchidas com espaços em branco.

In [282]:
dados.query("Charges_Total == ' '").shape

(11, 21)

Já na coluna `Churn` existe incosistência semelhante porém as células são vazias.

In [283]:
dados.Churn.value_counts()

No     5174
Yes    1869
        224
Name: Churn, dtype: int64

Na coluna `SeniorCitizen` os valores estão entre 0 e 1 quando deveriam estar como Yes ou No, matendo o padrão das demais colunas.

In [284]:
dados.SeniorCitizen.value_counts()

0    6085
1    1182
Name: SeniorCitizen, dtype: int64

## Tarefa 4 - Corrigir as incosistências nos dados

Para os dados faltantes na coluna `Charges_Total` irei transformá-los em NaN e os demais em float.

In [285]:
import numpy as np

# Substitui espaços em branco ' ' por NaN e converte os demais casos pra float64
dados.Charges_Total = dados.Charges_Total.apply(lambda x: np.nan if x == ' ' else float(x))

Verificando a quantidade de nulos em `Charges_Total`.

In [286]:
dados.Charges_Total.isna().sum()

11

De forma semelhante será feito na coluna de `Churn`.

In [287]:
# Caso haja valor vazio substitui por NaN caso contrário mantém o valor
dados.Churn = dados.Churn.apply(lambda x: np.nan if x == "" else x)

Verificando a quantidade de nulos em `Churn`.

In [288]:
dados.Churn.isna().sum()

224

Checando se existem valores vazios nas demais colunas.

In [289]:
for column in dados.columns:
    if len(dados.query(f'{column} == ""')):
        print(dados.query(f'{column} == ""'))
    if len(dados.query(f'{column} == " "')):
        print(dados.query(f'{column} == " "'))

Não há mais dados vazios.

Mudando os valores 0 e 1 da `SeniorCitizen' para Yes ou No

In [290]:
map_seniors = {0:'No', 1:'Yes'}
dados.SeniorCitizen.replace(map_seniors, inplace=True)

dados.SeniorCitizen.value_counts()

No     6085
Yes    1182
Name: SeniorCitizen, dtype: int64

## Tarefa 5 - Traduzir os dados

Primeiro passo será substituir as ocorrências de Yes e No por Sim e Não respectivamente

In [291]:
yes_no = {'Yes': 'Sim', 'No':'Não'}

colunas_yes_no = ['Churn', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService',
                     'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 
                     'StreamingTV', 'StreamingMovies', 'PaperlessBilling']

for coluna in colunas_yes_no:
    dados[coluna].replace(yes_no, inplace=True)

Na sequência é feita a tradução dos gêneros.

In [292]:
male_female = {'Male':'Masculino', 'Female':'Feminino'}

dados.gender.replace(male_female, inplace=True)
dados.gender.value_counts()

Masculino    3675
Feminino     3592
Name: gender, dtype: int64

Tradução dos registros sem serviço de telefone

In [293]:
dados.PhoneService.replace('No phone service', 'Sem serviço de telefone', inplace=True)
dados.MultipleLines.replace('No phone service', 'Sem serviço de telefone', inplace=True)

Tradução dos serviços de internet.

In [294]:
dados.InternetService.replace('Fiber optic', 'Fibra Óptica', inplace=True)

Tradução dos contratos.

In [295]:
contratos = {'Month-to-month' : 'Mensal', 'Two year': 'Dois anos', 'One year': 'Um ano'}
dados.Contract.replace(contratos, inplace=True)
dados.Contract.value_counts()

Mensal       4005
Dois anos    1743
Um ano       1519
Name: Contract, dtype: int64

Tradução dos métodos de pagamento.

In [296]:
metodo_pagamento = {'Electronic check': 'Cheque eletrônico', 'Mailed check':'Cheque',
                    'Bank transfer (automatic)': 'Transferência bancária',
                    'Credit card (automatic)': 'Cartão de crédito'}

dados.PaymentMethod.replace(metodo_pagamento, inplace=True)
dados.PaymentMethod.value_counts()

Cheque eletrônico         2445
Cheque                    1665
Transferência bancária    1589
Cartão de crédito         1568
Name: PaymentMethod, dtype: int64

Tradução dos registros sem serviço de internet.

In [297]:
sem_internet = {'No internet service': 'Sem serviço de internet'}
colunas = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
            'StreamingMovies']

for coluna in colunas:
    dados[coluna].replace(sem_internet, inplace=True)

Tradução das colunas

In [298]:
novos_nomes_colunas = ['ID_cliente', 'Churn', 'Genero', 'Senior', 'Parceiro', 'Dependentes', 'Tempo_permanencia',
                        'Servico_telefone', 'Multiplas_linhas', 'Servico_internet', 'Seguranca_online',
                        'Backup_online', 'Protecao_dispositivo', 'Suporte_tecnico', 'Streaming_TV',
                        'Streaming_filmes', 'Contrato', 'Fatura_online', 'Método_pagamento',
                        'Gastos_mensais', 'Gastos_totais']

dados.columns = novos_nomes_colunas
dados.head()

,ID_cliente,Churn,Genero,Senior,Parceiro,Dependentes,Tempo_permanencia,Servico_telefone,Multiplas_linhas,Servico_internet,Seguranca_online,Backup_online,Protecao_dispositivo,Suporte_tecnico,Streaming_TV,Streaming_filmes,Contrato,Fatura_online,Método_pagamento,Gastos_mensais,Gastos_totais
0,0002-ORFBO,Não,Feminino,Não,Sim,Sim,9,Sim,Não,DSL,Não,Sim,Não,Sim,Sim,Não,Um ano,Sim,Cheque,65.6,593.30
1,0003-MKNFE,Não,Masculino,Não,Não,Não,9,Sim,Sim,DSL,Não,Não,Não,Não,Não,Sim,Mensal,Não,Cheque,59.9,542.40
2,0004-TLHLJ,Sim,Masculino,Não,Não,Não,4,Sim,Não,Fibra Óptica,Não,Não,Sim,Não,Não,Não,Mensal,Sim,Cheque eletrônico,73.9,280.85
3,0011-IGKFF,Sim,Masculino,Sim,Sim,Não,13,Sim,Não,Fibra Óptica,Não,Sim,Sim,Não,Sim,Sim,Mensal,Sim,Cheque eletrônico,98.0,1237.85
4,0013-EXCHZ,Sim,Feminino,Sim,Sim,Não,3,Sim,Não,Fibra Óptica,Não,Não,Não,Sim,Sim,Não,Mensal,Sim,Cheque,83.9,267.40


In [299]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID_cliente            7267 non-null   object 
 1   Churn                 7043 non-null   object 
 2   Genero                7267 non-null   object 
 3   Senior                7267 non-null   object 
 4   Parceiro              7267 non-null   object 
 5   Dependentes           7267 non-null   object 
 6   Tempo_permanencia     7267 non-null   int64  
 7   Servico_telefone      7267 non-null   object 
 8   Multiplas_linhas      7267 non-null   object 
 9   Servico_internet      7267 non-null   object 
 10  Seguranca_online      7267 non-null   object 
 11  Backup_online         7267 non-null   object 
 12  Protecao_dispositivo  7267 non-null   object 
 13  Suporte_tecnico       7267 non-null   object 
 14  Streaming_TV          7267 non-null   object 
 15  Streaming_filmes     

## Tarefa 6 - Criar coluna de contas diárias

In [300]:
dados['Gastos_diarios'] = dados.Gastos_mensais / 30

In [301]:
dados.head()

,ID_cliente,Churn,Genero,Senior,Parceiro,Dependentes,Tempo_permanencia,Servico_telefone,Multiplas_linhas,Servico_internet,Seguranca_online,Backup_online,Protecao_dispositivo,Suporte_tecnico,Streaming_TV,Streaming_filmes,Contrato,Fatura_online,Método_pagamento,Gastos_mensais,Gastos_totais,Gastos_diarios
0,0002-ORFBO,Não,Feminino,Não,Sim,Sim,9,Sim,Não,DSL,Não,Sim,Não,Sim,Sim,Não,Um ano,Sim,Cheque,65.6,593.30,2.186667
1,0003-MKNFE,Não,Masculino,Não,Não,Não,9,Sim,Sim,DSL,Não,Não,Não,Não,Não,Sim,Mensal,Não,Cheque,59.9,542.40,1.996667
2,0004-TLHLJ,Sim,Masculino,Não,Não,Não,4,Sim,Não,Fibra Óptica,Não,Não,Sim,Não,Não,Não,Mensal,Sim,Cheque eletrônico,73.9,280.85,2.463333
3,0011-IGKFF,Sim,Masculino,Sim,Sim,Não,13,Sim,Não,Fibra Óptica,Não,Sim,Sim,Não,Sim,Sim,Mensal,Sim,Cheque eletrônico,98.0,1237.85,3.266667
4,0013-EXCHZ,Sim,Feminino,Sim,Sim,Não,3,Sim,Não,Fibra Óptica,Não,Não,Não,Sim,Sim,Não,Mensal,Sim,Cheque,83.9,267.40,2.796667


## Observações finais.

Existem 224 registros de Churn nulo e 11 registros de Gastos_totais nulos.

In [302]:
dados.Churn.isna().sum()

224

In [303]:
dados.Gastos_totais.isna().sum()

11

Exportando arquivo csv

In [304]:
dados.to_csv('dados_clientes_alura_voz.csv', index=False)